In [1]:
source("aux_2.R")

Loading required package: NLP

Loading required package: koRpus.lang.en

Loading required package: koRpus

Loading required package: sylly

For information on available language packages for 'koRpus', run

  available.koRpus.lang()

and see ?install.koRpus.lang()



Attaching package: ‘koRpus’


The following object is masked from ‘package:tm’:

    readTagged



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
dataset <- read.csv("train.csv", col.names = c("Label", "Text", "Tag"))
classes <- as.integer(sort(unique(dataset$Label)))
args <- sort(unique(unlist(strsplit(dataset$Tag, ","))))

In [3]:
dataset$Text <- clean(dataset$Text)
dataset <- clean_empty_rows(dataset)
dataset <- dataset[!is.na(dataset$Text), ]

In [4]:
occ_threshold <- 2
vocabulary <- get_vocabulary(dataset$Text, occ_threshold)

In [5]:
seventy_percent <- as.integer(length(dataset$Text) * 0.7)
eightyfive_percent <- as.integer(length(dataset$Text) * 0.85)
hundred_percent <- length(dataset$Text)

train_set <- dataset[1:seventy_percent, ]
val_set <- dataset[(seventy_percent + 1):eightyfive_percent, ]
test_set <- dataset[(eightyfive_percent + 1):hundred_percent, ]

In [6]:
val_set <- val_set[!is.na(val_set$Text), ]

In [7]:
length(train_set[is.na(train_set$Text), ]$Text)

[1] 0

In [8]:
occ_threshold <- 1
model <- train_multinomial_nb(classes, train_set, occ_threshold)
#print(model$vocab)
#print(model$prior)
#print(model$condprob)

In [9]:
docs <- val_set$Text

pred_labels <- sapply(docs, function(doc) {
  apply_multinomial_nb(classes, model$vocab, model$prior, model$condprob, doc)
})

In [10]:
correct_predictions <- sum(val_set$Label == pred_labels)
total_predictions <- length(val_set$Label)
accuracy <- correct_predictions / total_predictions

cat("Accuracy:", accuracy)

Accuracy: 0.2259115